# Useful resources

http://stackoverflow.com/questions/9401658/matplotlib-animating-a-scatter-plot

https://plot.ly/python/streaming-bubbles-tutorial/

https://jakevdp.github.io/blog/2012/08/18/matplotlib-animation-tutorial/

In [3]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import pandas as pd
import seaborn as sns

In [4]:
np.set_printoptions(precision=2)

In [5]:
df=pd.read_pickle('../datasets/df_userID.pickle')

In [6]:
df['Age']=df.RaceYear-df.Year

In [7]:
df=df[df.Age>4]

In [8]:
df=df[df.Age<100]

In [9]:
categories=[0,5,10,15.5,25,43]

In [10]:
mens=df[df.Sex=='M']

In [11]:
df_categories=[df[(df.Distance>categories[i-1]) &(df.Distance<categories[i])] for i in range(1,len(categories))]

In [12]:
[x.shape[0] for x in df_categories]

[265021, 312078, 188205, 365711, 159084]

In [13]:
ages=list(range(4,81,4))

In [14]:
df_categories_age=[[] for i in df_categories]

In [15]:
for j in range(len(categories)-1):
    df_category=df_categories[j]
    df_categories_age[j]=[df_category[(df_category.Age>ages[i-1]) &(df_category.Age<ages[i])] for i in range(1,len(ages))]

In [16]:
df_categories_age_size=[[0 for i in range(len(df_categories_age[0]))] for i in range(len(df_categories_age))]

In [17]:
for i in range(len(df_categories_age)):
    for j in range(len(df_categories_age[0])):
        df_categories_age_size[i][j]=df_categories_age[i][j].shape[0]
df_categories_age_size=np.array(df_categories_age_size)

In [18]:
df_categories_age_size

array([[51470, 97168, 35913,  3719,  1283,  1600,  1269,   964,  1126,
         1398,  1249,  1165,   867,   543,   571,   334,   173,    81,
           22],
       [   11,   368,  2461, 10017, 13152, 19829, 22938, 23808, 26644,
        30681, 26645, 21727, 14599,  9073,  5846,  3326,  1766,   761,
          251],
       [    0,    36,   669,  3969,  5165,  8241, 11119, 12509, 15317,
        21929, 21254, 14740,  9464,  6245,  4612,  2640,  1542,   643,
          205],
       [    5,   105,   784,  5945, 11224, 20594, 27413, 30666, 35750,
        40333, 36520, 27038, 16894,  9984,  5796,  2955,  1370,   469,
          133],
       [    0,     2,    17,   647,  2500,  5497,  9541, 12457, 15965,
        19686, 18329, 14857,  9519,  5489,  2891,  1393,   504,   132,
           34]])

In [19]:
df_category_age_usercount=[[y.groupby(y.UserID).count().shape[0] for y in x] for x in df_categories_age]
df_age_usercount=np.sum(df_category_age_usercount,axis=0)

In [20]:
df_percentage_of_users_per_category=df_category_age_usercount/df_age_usercount

In [21]:
a=df_categories_age[2][0]

In [22]:
final_table=[[np.mean(y.groupby(y.UserID).count().Race) for y in x] for x in df_categories_age]

In [23]:
np.array(final_table)

array([[ 1.46,  1.62,  1.53,  1.14,  1.19,  1.21,  1.17,  1.16,  1.22,
         1.23,  1.29,  1.28,  1.41,  1.46,  1.5 ,  1.48,  1.59,  1.69,
         1.57],
       [ 1.  ,  1.26,  1.3 ,  1.42,  1.41,  1.44,  1.46,  1.53,  1.6 ,
         1.69,  1.75,  1.82,  1.86,  1.89,  1.95,  2.  ,  2.12,  2.33,
         2.09],
       [  nan,  1.24,  1.6 ,  1.73,  1.63,  1.62,  1.66,  1.68,  1.76,
         1.99,  2.06,  1.92,  1.93,  2.  ,  2.2 ,  2.27,  2.6 ,  2.66,
         2.81],
       [ 1.  ,  1.36,  1.3 ,  1.29,  1.32,  1.4 ,  1.43,  1.49,  1.57,
         1.69,  1.75,  1.79,  1.78,  1.82,  1.83,  1.84,  1.92,  1.91,
         1.96],
       [  nan,  1.  ,  1.  ,  1.14,  1.21,  1.28,  1.33,  1.36,  1.42,
         1.48,  1.5 ,  1.52,  1.55,  1.55,  1.57,  1.6 ,  1.64,  1.59,  1.7 ]])

In [24]:
paces_matrix=[[0 for j in range(len(ages)-1)] for i in range(len(categories)-1)]

In [25]:
for i in range(len(categories)-1):
    for j in range(len(ages)-1):
        paces_matrix[i][j]=np.mean(df_categories_age[i][j].Pace)

In [26]:
np_paces_matrix=np.array(paces_matrix)

In [27]:
np_paces=np.zeros(np_paces_matrix.shape)

In [28]:
for i in range(np_paces_matrix.shape[0]):
    for j in range(np_paces_matrix.shape[1]):
        np_paces[i,j]=np_paces_matrix[i,j].total_seconds()

In [29]:
def interpolate(matrix,interpolation=10):
    return_matrix=np.zeros([matrix.shape[0],(matrix.shape[1]-1)*interpolation+1])
    
    return_matrix[:,0]=matrix[:,0]
    starting_index=0
    for j in range(1,matrix.shape[1]):
        gradient=(matrix[:,j]-matrix[:,j-1])/interpolation
        for i in range(1,interpolation):
            return_matrix[:,starting_index+i]=return_matrix[:,starting_index]+i*gradient
        return_matrix[:,starting_index+interpolation]=matrix[:,j]
        starting_index+=interpolation
    return return_matrix

In [39]:
df_categories_age_size=interpolate(df_categories_age_size[:,1:])

In [40]:
np_paces=interpolate(np_paces[:,1:])

In [41]:
df_percentage_of_users_per_category=interpolate(df_percentage_of_users_per_category[:,1:])

In [42]:
df_percentage_of_users_per_category.shape

(5, 1691)

In [43]:
df_categories_age_size.shape

(5, 1691)

In [44]:
np_paces.shape

(5, 1691)

In [45]:
np_paces

array([[ 287.63,  287.46,  287.29, ...,  461.85,  462.4 ,  462.95],
       [ 324.49,  324.3 ,  324.1 , ...,  402.71,  403.  ,  403.29],
       [ 343.46,  343.31,  343.15, ...,  399.97,  400.23,  400.5 ],
       [ 466.44,  465.1 ,  463.77, ...,  414.62,  414.74,  414.86],
       [ 424.44,  424.09,  423.73, ...,  465.99,  465.96,  465.94]])

In [46]:
distances_list=['<5km','5km','10km','21.0975km','42.195km']

In [69]:
class AnimatedScatter(object):
    """An animated scatter plot using matplotlib.animations.FuncAnimation."""
    def __init__(self, numpoints=50):
        self.numpoints = numpoints
        self.stream = self.data_stream()

        # Setup the figure and axes...
        self.fig, self.ax = plt.subplots()
        # Then setup FuncAnimation.
        self.ani = animation.FuncAnimation(self.fig, self.update, interval=10, 
                                           init_func=self.setup_plot, blit=True)

    def setup_plot(self):
        """Initial drawing of the scatter plot."""
        x,y,s,time_step = next(self.stream)
        y = [r for r in y]
        x = [i for i in x]
#         s = [1000 for i in y]
        c = 1
#         plt.annotate('ciao',(0.4,300))
#         print(y)
#         print(s)
        self.scat = self.ax.scatter(x, y, s=s, animated=True)
        self.ax.axis([0, 1, 250, 500])

        # For FuncAnimation's sake, we need to return the artist we'll be using
        # Note that it expects a sequence of artists, thus the trailing comma.
        return self.scat,

    def data_stream(self):
        """Generate a random walk (brownian motion). Data is scaled to produce
        a soft "flickering" effect."""
        x=df_percentage_of_users_per_category
        data=np_paces
        size=df_categories_age_size
        size=np.sqrt(size)*10
        
        for i in range(1,data.shape[1]):
            yield x[:,i],data[:,i],size[:,i],i
        
        #         data = np.random.random((4, self.numpoints))
#         xy = data[:2, :]
#         s, c = data[2:, :]
#         xy -= 0.5
#         xy *= 10
#         for i in range(1000):
#             xy += 0.03 * (np.random.random((2, self.numpoints)) - 0.5)
#             s += 0.05 * (np.random.random(self.numpoints) - 0.5)
#             c += 0.02 * (np.random.random(self.numpoints) - 0.5)
#             yield data

    def update(self, i):
        """Update the scatter plot."""
        try:
            x,y,s,time_step = next(self.stream)
        except:
            return self.scat
        y = [r for r in y]
        x = [i for i in x]
#         s = [1000 for i in y]
        c = 1
#         print(y)
#         print(s)
        graph=np.row_stack((y,x))
        # Set x and y data...
        self.scat.set_offsets(graph)
        # Set sizes...
        self.scat._sizes = s# 300 * abs(data[2])**1.5 + 100

        # Set colors..
#         self.scat.set_array(data[3])
        ann=[]
        self.scat =self.ax.scatter(x, y, s=s, animated=True)
        ann.append(self.scat)
        for i in range(len(x)):
            ann.append(plt.annotate(distances_list[i],(x[i]+0.03,y[i]+5),weight='bold',fontname='Helvetica',color='red'))
            
        
        
        # We need to return the updated artist for FuncAnimation to draw..
        # Note that it expects a sequence of artists, thus the trailing comma.
        return ann

    def show(self):
        plt.show()


In [70]:
a = AnimatedScatter()

a.show()

# Old code

In [ ]:
class AnimatedScatter(object):
    """An animated scatter plot using matplotlib.animations.FuncAnimation."""
    def __init__(self, numpoints=50):
        self.numpoints = numpoints
        self.stream = self.data_stream()

        # Setup the figure and axes...
        self.fig, self.ax = plt.subplots()
        # Then setup FuncAnimation.
        self.ani = animation.FuncAnimation(self.fig, self.update, interval=5, 
                                           init_func=self.setup_plot, blit=True)

    def setup_plot(self):
        """Initial drawing of the scatter plot."""
        x, y, s, c = next(self.stream)
        self.scat = self.ax.scatter(x, y, c=c, s=s, animated=True)
        self.ax.axis([-1, 1, -10, 10])

        # For FuncAnimation's sake, we need to return the artist we'll be using
        # Note that it expects a sequence of artists, thus the trailing comma.
        return self.scat,

    def data_stream(self):
        """Generate a random walk (brownian motion). Data is scaled to produce
        a soft "flickering" effect."""
        data=np_paces_matrix
        
        for i in range(1,data.shape[1]):
            yield data[:,i]
        
        #         data = np.random.random((4, self.numpoints))
#         xy = data[:2, :]
#         s, c = data[2:, :]
#         xy -= 0.5
#         xy *= 10
#         for i in range(1000):
#             xy += 0.03 * (np.random.random((2, self.numpoints)) - 0.5)
#             s += 0.05 * (np.random.random(self.numpoints) - 0.5)
#             c += 0.02 * (np.random.random(self.numpoints) - 0.5)
#             yield data

    def update(self, i):
        """Update the scatter plot."""
        data = next(self.stream)

        # Set x and y data...
        self.scat.set_offsets(data[:2, :])
        # Set sizes...
        self.scat._sizes = 300 * abs(data[2])**1.5 + 100
        # Set colors..
        self.scat.set_array(data[3])

        # We need to return the updated artist for FuncAnimation to draw..
        # Note that it expects a sequence of artists, thus the trailing comma.
        return self.scat,

    def show(self):
        plt.show()
